In [1]:
#https://towardsdatascience.com/how-to-create-a-chatbot-with-python-deep-learning-in-less-than-an-hour-56a063bdfc44
#import libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Navya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Navya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

Using TensorFlow backend.


In [3]:
words=[]
classes=[]
documents=[]
ignore_words=['?','!']
data_file=open('intents.json').read()
intents=json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #take each word and tokenize it
        w=nltk.word_tokenize(pattern)
        words.extend(w)
        #adding documents
        documents.append((w,intent['tag']))
        
        #adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
words=[lemmatizer.lemmatize(w.lower())for w in words if w not in ignore_words]
words=sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents),"documents")

print(len(classes),"classes",classes)

print(len(words),"unique lemmatized words",words)

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [6]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [7]:
#initializing training data
training=[]
output_empty=[0]*len(classes)

for doc in documents:
    #initialise bag of words
    bag=[]
    #list of tokenized words for the pattern
    pattern_words=doc[0]
    #lemmatize each word - create a base word, in attempt to represent related words
    pattern_words=[lemmatizer.lemmatize(word.lower())for word in pattern_words]
    
    #create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    #output is 0 for each tag and 1 for current tag(for each pattern)
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    
    training.append([bag,output_row])

In [8]:
#shuffle our features and turn into np.array
random.shuffle(training)
training=np.array(training)

#create train and test lists. X-patterns, Y-intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [9]:
#create model 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
#equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

W0213 20:30:41.021028 10032 deprecation_wrapper.py:119] From C:\Users\Navya\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0213 20:30:41.600654 10032 deprecation_wrapper.py:119] From C:\Users\Navya\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0213 20:30:41.712217 10032 deprecation_wrapper.py:119] From C:\Users\Navya\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0213 20:30:41.894385 10032 deprecation_wrapper.py:119] From C:\Users\Navya\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0213 20:30:41.925284 10032 deprecation.py:506

In [10]:
#compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd=SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

W0213 20:40:53.110333 10032 deprecation_wrapper.py:119] From C:\Users\Navya\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0213 20:40:53.201785 10032 deprecation_wrapper.py:119] From C:\Users\Navya\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [11]:
#fitting and saving the model
hist=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1)
model.save('chatbot_model.h5',hist)

print("model created")

W0213 20:43:41.033987 10032 deprecation.py:323] From C:\Users\Navya\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/200
47/47 [==============================] - 2s 38ms/step - loss: 2.1716 - acc: 0.0851
Epoch 2/200
47/47 [==============================] - 0s 509us/step - loss: 2.1635 - acc: 0.1915
Epoch 3/200
47/47 [==============================] - 0s 548us/step - loss: 2.0753 - acc: 0.2128
Epoch 4/200
47/47 [==============================] - 0s 480us/step - loss: 1.9416 - acc: 0.3191
Epoch 5/200
47/47 [==============================] - 0s 521us/step - loss: 1.9370 - acc: 0.3617
Epoch 6/200
47/47 [==============================] - 0s 356us/step - loss: 1.7367 - acc: 0.5106
Epoch 7/200
47/47 [==============================] - 0s 510us/step - loss: 1.6240 - acc: 0.6170
Epoch 8/200
47/47 [==============================] - 0s 445us/step - loss: 1.5967 - acc: 0.5745
Epoch 9/200
47/47 [==============================] - 0s 468us/step - loss: 1.3651 - acc: 0.5745
Epoch 10/200
47/47 [==============================] - 0s 530us/step - loss: 1.3169 - acc: 0.7234
Epoch 11/200
47/47 [====================